In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
print("hello")

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
sys.path.append(os.path.abspath(os.path.join(".", "src")))

from audio_processing import audio_import

In [ ]:
loader = audio_import.AudioLoader()

print(loader.directory)
#ensure files can be viewed
os.listdir(loader.directory)

In [ ]:
# Compare ex5 (original) vs ex5_distorted side by side (waveforms)
import os
import numpy as np
import matplotlib.pyplot as plt
from audio_processing import audio_import

loader = audio_import.AudioLoader()

orig_name = 'ex5.wav'
dist_name = 'ex5_distorted.wav'

# Check file existence in loader directory
orig_path = os.path.join(loader.directory, orig_name)
dist_path = os.path.join(loader.directory, dist_name)
if not os.path.exists(orig_path):
    print(f"Missing: {orig_name} in {loader.directory}")
if not os.path.exists(dist_path):
    print(f"Missing: {dist_name} in {loader.directory}")

# Load both files
pair = loader.load_batch([orig_name, dist_name])

y_orig = pair[orig_name]['data']
sr_orig = pair[orig_name]['samplerate']

y_dist = pair[dist_name]['data']
sr_dist = pair[dist_name]['samplerate']

# Warn if sample rates differ
if sr_orig != sr_dist:
    print(f"Warning: sample rates differ (orig={sr_orig}, dist={sr_dist}). Time axes will reflect their own SRs.")

# Time axes
T_orig = len(y_orig) / sr_orig
T_dist = len(y_dist) / sr_dist

t_orig = np.linspace(0, T_orig, len(y_orig))
t_dist = np.linspace(0, T_dist, len(y_dist))

# Unified y-limits for fair comparison
amp = max(np.max(np.abs(y_orig)), np.max(np.abs(y_dist)))
ymin, ymax = -1.05 * amp, 1.05 * amp

fig, axs = plt.subplots(1, 2, figsize=(14, 4), sharey=True)

axs[0].plot(t_orig, y_orig, color='steelblue', linewidth=0.6)
axs[0].set_title('ex5 (original)')
axs[0].set_xlabel('Time (s)')
axs[0].set_ylabel('Amplitude')
axs[0].grid(True, alpha=0.3)
axs[0].set_ylim(ymin, ymax)

axs[1].plot(t_dist, y_dist, color='crimson', linewidth=0.6)
axs[1].set_title('ex5_distorted')
axs[1].set_xlabel('Time (s)')
axs[1].grid(True, alpha=0.3)
axs[1].set_ylim(ymin, ymax)

plt.suptitle('Waveform comparison: ex5 vs ex5_distorted', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# test artificial distortion
print(os.listdir(os.path.join("..", "src", "audio_processing")))

from audio_processing.artifact_simulate import ArtifactSim

aSim = ArtifactSim()

artifacts = {}

artifacts['ex5_distorted.wav'] = aSim.distort_audio('ex5.wav', 'ex5_distorted.wav')
artifacts['ex6_distorted.wav'] = aSim.distort_audio('ex6.wav', 'ex6_distorted.wav')

In [ ]:
loader = audio_import.AudioLoader()

data = loader.load_batch(['ex5_distorted.wav', 'ex6_distorted.wav'])

for name, info in data.items():
    print(f"{name}: {info['data'].shape}, {info['samplerate']} Hz")

In [ ]:
# visualize as sound wave with artifacts highlighted
for name, info in data.items():
    plt.figure(figsize=(12, 4))
    
    # Create time axis in seconds
    duration = len(info['data']) / info['samplerate']
    time_axis = np.linspace(0, duration, len(info['data']))
    
    # Plot waveform
    plt.plot(time_axis, info['data'], alpha=0.7, linewidth=0.5, color='blue')
    
    # Highlight artifacts if available
    if name in ['ex5_distorted.wav', 'ex6_distorted.wav']:
        # artifacts is a list of tuples: (type, timestamp_in_seconds, duration_ms)
        if 'artifacts' in locals() or 'artifacts' in globals():
            for artifact_data in artifacts[name]:
                artifact_type = artifact_data[0]
                timestamp = artifact_data[1]
                duration_ms = artifact_data[2] if len(artifact_data) > 2 else 10
                duration_sec = duration_ms / 1000  # Convert ms to seconds
                
                color_map = {
                    'click': 'red',
                    'pop': 'orange', 
                    'cutout': 'purple',
                    'clipping': 'yellow'
                }
                color = color_map.get(artifact_type, 'gray')
                
                # Highlight the region with axvspan
                plt.axvspan(timestamp, timestamp + duration_sec, 
                           color=color, alpha=0.3, label=artifact_type)
    
    plt.title(f"{name} waveform with artifacts highlighted")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude")
    plt.grid(True, alpha=0.3)
    
    # Add legend (remove duplicates)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='upper right')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# visualize as spectrogram with artifacts highlighted
import numpy as np
import librosa.display

for name, info in data.items():
    plt.figure(figsize=(12, 5))
    
    # Compute the spectrogram
    S = np.abs(librosa.stft(info['data']))
    S_db = librosa.amplitude_to_db(S, ref=np.max)
    librosa.display.specshow(S_db, sr=info['samplerate'], x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    
    # Highlight artifacts if available
    if name in ['ex5_distorted.wav', 'ex6_distorted.wav']:
        if 'artifacts' in locals() or 'artifacts' in globals():
            for artifact_data in artifacts[name]:
                artifact_type = artifact_data[0]
                timestamp = artifact_data[1]
                duration_ms = artifact_data[2] if len(artifact_data) > 2 else 10
                duration_sec = duration_ms / 1000  # Convert ms to seconds
                
                # Brighter colors with better contrast for spectrogram
                color_map = {
                    'click': '#FF3366',      # Bright pink/red
                    'pop': '#FFD700',        # Bright gold
                    'cutout': '#00FFFF',     # Cyan
                    'clipping': '#00FF00'    # Bright green
                }
                color = color_map.get(artifact_type, 'white')
                
                # Highlight the region with axvspan (higher alpha for visibility)
                plt.axvspan(timestamp, timestamp + duration_sec, 
                           color=color, alpha=0.35, label=artifact_type)
                # Add a brighter vertical line at the start
                plt.axvline(x=timestamp, color=color, alpha=0.9, 
                           linestyle='--', linewidth=2)
    
    plt.title(f"{name} Spectrogram with artifacts highlighted")
    
    # Add legend (remove duplicates) with darker background for visibility
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    if by_label:
        legend = plt.legend(by_label.values(), by_label.keys(), loc='upper right')
        legend.get_frame().set_facecolor('black')
        legend.get_frame().set_alpha(0.8)
        for text in legend.get_texts():
            text.set_color('white')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Detect and visualize clipping and cutout regions on waveform
from audio_processing import distortion_detection as dd
from audio_processing.utils import seconds_to_mmss
import matplotlib.pyplot as plt
import numpy as np

for name, info in data.items():
    audio = info['data']
    sr = info['samplerate']

    # Run detectors
    clipping_regions = dd.detect_clipping(audio, sr, threshold=0.98, min_clip_duration_ms=10)
    cutout_regions = dd.detect_cutout(audio, sr, threshold=0.001, min_silence_duration_ms=50)

    print(f"\n{name}")
    if clipping_regions:
        print("  Clipping regions:")
        for start, end in clipping_regions:
            print(f"- {seconds_to_mmss(start)}  - {seconds_to_mmss(end)}")
    else:
        print("  No clipping detected.")
    if cutout_regions:
        print("  Cutout regions:")
        for start, end in cutout_regions:
            print(f"- {seconds_to_mmss(start)}  - {seconds_to_mmss(end)}")
    else:
        print("  No cutouts detected.")

    # Plot waveform with overlays
    duration = len(audio) / sr
    t = np.linspace(0, duration, len(audio))
    plt.figure(figsize=(12, 4))
    plt.plot(t, audio, color='steelblue', linewidth=0.6, label='waveform')

    # Overlay clipping regions (yellow)
    for (start, end) in clipping_regions:
        plt.axvspan(start, end, color='yellow', alpha=0.35, label='clipping')
    # Overlay cutout regions (purple)
    for (start, end) in cutout_regions:
        plt.axvspan(start, end, color='purple', alpha=0.3, label='cutout')

    plt.title(f"{name} - Waveform with detected clipping and cutouts")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True, alpha=0.3)

    # Deduplicate legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    if by_label:
        plt.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.tight_layout()
    plt.show()